Input: Comerturas KML de la Capa "Uso"
Output: Dataframe

In [4]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd

# Lista con las rutas de todos los archivos KML
kml_files = ['usolaspalmas.kml','usolospinos.kml']  

# Lista para almacenar las características de todos los terrenos
terrenos = []

# Función para procesar un archivo KML
def procesar_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Espacio de nombres KML
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Iterar sobre cada <Placemark>
    placemarks = root.findall('.//kml:Placemark', ns)
    for placemark in placemarks:
        # Extraer el nombre y descripción
        nombre = placemark.find('kml:name', ns).text
        descripcion_html = placemark.find('kml:description', ns).text

        # Procesar la descripción para extraer la segunda tabla
        soup = BeautifulSoup(descripcion_html, 'html.parser')
        tables = soup.find_all('table')
        second_table = tables[1] if len(tables) > 1 else None

        # Almacenar los pares clave-valor de la segunda tabla
        info_terreno = {'nombre': nombre}
        if second_table:
            rows = second_table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:  # Esperamos pares clave-valor
                    key = cols[0].text.strip()
                    value = cols[1].text.strip()
                    info_terreno[key] = value

        # Extraer las coordenadas (múltiples puntos delimitan el terreno)
        coordenadas_texto = placemark.find('.//kml:coordinates', ns).text.strip()
        coordenadas = [tuple(map(float, coord.split(','))) for coord in coordenadas_texto.split()]
        info_terreno['coordenadas'] = coordenadas

        terrenos.append(info_terreno)

# Procesar todos los archivos KML
for kml_file in kml_files:
    procesar_kml(kml_file)

# Convertir toda la información a un DataFrame
df_terrenos = pd.DataFrame(terrenos)

# Exportar el DataFrame a Excel
df_terrenos.to_excel('terrenos.xlsx', index=False)

# Exportar el DataFrame a CSV
df_terrenos.to_csv('terrenos.csv', index=False)

print("Datos exportados exitosamente a Excel y CSV.")


Datos exportados exitosamente a Excel y CSV.


In [5]:
df_terrenos.describe()

,nombre,FID,ID_UNICO,ID_PREDIO,NOMBRE,ID_USO,ID_SUBUSO,ID_ESTRUC,ID_TIFO,ID_STIF,...,ID_MANEJO,ID_RALEO,ID_PODA,ID_SANID,ID_ROTAC,USO_COT,USO_CEFOR,OBS,SUP_HA,coordenadas
count,580,580,580,580,580,580,580,580,580,580,...,580,580,580,580,580,580,580,580,580,580
unique,2,318,318,2,2,5,6,5,3,5,...,4,6,5,4,4,15,10,19,248,580
top,01,0,1,01,Los Pinos,04,01,01,,,...,,,,,,Plantacion Adulta,Plantacion,,0.42,"[(-73.14953102003405, -39.74768325608614, 0.0)..."
freq,318,2,2,318,318,427,292,220,463,463,...,378,366,366,366,368,219,258,549,12,1


In [6]:
# prompt: nombres unicos de la columna USO_CEFOR

usos_cefor_unicos = df_terrenos['USO_CEFOR'].unique()
print(usos_cefor_unicos)


['Faja Camino' 'Bosques Mixtos' 'Plantacion' 'Bosque Nativo' 'Matorral'
 'Bosque Exoticas Asilvestradas' 'Reforestacion' 'Pradera'
 'Terrenos Humedos' 'Cuerpos de Agua']


In [1]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import json
import os

# KML de entrada
kml_files = ['usolaspalmas.kml']
terrenos = []

def procesar_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    placemarks = root.findall('.//kml:Placemark', ns)
    for placemark in placemarks:
        nombre = placemark.find('kml:name', ns).text
        descripcion_html = placemark.find('kml:description', ns).text
        soup = BeautifulSoup(descripcion_html, 'html.parser')
        tables = soup.find_all('table')
        second_table = tables[1] if len(tables) > 1 else None

        info_terreno = {'nombre': nombre}
        if second_table:
            for row in second_table.find_all('tr'):
                cols = row.find_all('td')
                if len(cols) == 2:
                    key = cols[0].text.strip().upper()
                    value = cols[1].text.strip()
                    info_terreno[key] = value

        # Extraer coordenadas (y eliminar altitud)
        coords_raw = placemark.find('.//kml:coordinates', ns).text.strip()
        coords = [(float(c.split(',')[0]), float(c.split(',')[1])) for c in coords_raw.split()]
        
        # Cerrar polígono si es necesario
        if coords[0] != coords[-1]:
            coords.append(coords[0])
        
        info_terreno['coordenadas'] = coords

        # Generar ID combinado
        id_predio = info_terreno.get("ID_PREDIO", "PNA")
        id_rodal = info_terreno.get("ID_UNICO", "RNA")  # o 'ID_RODAL'
        info_terreno["ID"] = str(id_predio) + "_" + str(id_rodal)

        terrenos.append(info_terreno)

# Procesar KML
for file in kml_files:
    procesar_kml(file)

# Convertir a DataFrame
df = pd.DataFrame(terrenos)

# Exportar Excel y CSV
df.to_excel("terrenos67.xlsx", index=False)
#df.to_csv("terrenos.csv", index=False)

print("Datos procesados:", df.shape)

# Crear GeoJSON
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "id": row["ID"],  # ID único
        "properties": {
            "ID_ESP1": row.get("ID_ESP1", None),  # u otra propiedad que necesites
            "nombre": row["nombre"]
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [row["coordenadas"]]
        }
    }
    geojson_data["features"].append(feature)

# Guardar GeoJSON
with open("terrenos67.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson_data, f, ensure_ascii=False, indent=2)

print("GeoJSON exportado exitosamente.")

Datos procesados: (262, 30)
GeoJSON exportado exitosamente.


In [3]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import json
from shapely.geometry import Polygon

# KML de entrada
kml_files = ['usolaspalmas.kml']
terrenos = []

def cerrar_poligono(coords):
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    return coords

def procesar_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    placemarks = root.findall('.//kml:Placemark', ns)
    for placemark in placemarks:
        nombre = placemark.find('kml:name', ns).text
        descripcion_html = placemark.find('kml:description', ns).text
        soup = BeautifulSoup(descripcion_html, 'html.parser')
        tables = soup.find_all('table')
        second_table = tables[1] if len(tables) > 1 else None

        info_terreno = {'nombre': nombre}
        if second_table:
            for row in second_table.find_all('tr'):
                cols = row.find_all('td')
                if len(cols) == 2:
                    key = cols[0].text.strip().upper()
                    value = cols[1].text.strip()
                    info_terreno[key] = value

        # Extraer coordenadas (eliminar altitud)
        coords_raw = placemark.find('.//kml:coordinates', ns).text.strip()
        coords = [(float(c.split(',')[0]), float(c.split(',')[1])) for c in coords_raw.split()]
        coords = cerrar_poligono(coords)

        # Simplificar el polígono
        polygon = Polygon(coords)
        simplified_polygon = polygon.simplify(0.0005, preserve_topology=True)
        simplified_coords = list(simplified_polygon.exterior.coords)

        info_terreno['coordenadas'] = simplified_coords

        # Generar ID combinado
        id_predio = info_terreno.get("ID_PREDIO", "PNA")
        id_rodal = info_terreno.get("ID_UNICO", "RNA")
        info_terreno["ID"] = str(id_predio) + "_" + str(id_rodal)

        terrenos.append(info_terreno)

# Procesar archivos
for file in kml_files:
    procesar_kml(file)

# Exportar CSV/Excel
df = pd.DataFrame(terrenos)
df.to_excel("terrenos.xlsx", index=False)
df.to_csv("terrenos.csv", index=False)

# Crear GeoJSON
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "id": row["ID"],
        "properties": {
            "ID_ESP1": row.get("ID_ESP1", None),
            "nombre": row["nombre"]
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [row["coordenadas"]]
        }
    }
    geojson_data["features"].append(feature)

# Guardar GeoJSON
with open("terrenos_simplificado.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson_data, f, ensure_ascii=False, indent=2)

print("GeoJSON simplificado exportado exitosamente.")


GeoJSON simplificado exportado exitosamente.


codigo simplificado. 6 de julio 21.24

In [2]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import json
from shapely.geometry import Polygon
from shapely.errors import TopologicalError

# Ruta al archivo KML
kml_files = ['usolaspalmas.kml','usolospinos.kml']

# Cargar tabla de especies desde Excel
df_especies = pd.read_excel("Terrenos (1).xlsx", sheet_name="Hoja1",keep_default_na=False)
df_especies = df_especies.reset_index().rename(columns={"ID": "ID_ESPECIE_GLOBAL"})

# Lista para almacenar los terrenos
terrenos = []

def cerrar_poligono(coords):
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    return coords

def procesar_kml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    for placemark in root.findall('.//kml:Placemark', ns):
        nombre = placemark.find('kml:name', ns).text
        descripcion_html = placemark.find('kml:description', ns).text
        soup = BeautifulSoup(descripcion_html, 'html.parser')
        tables = soup.find_all('table')
        tabla = tables[1] if len(tables) > 1 else None

        info = {'nombre': nombre}
        if tabla:
            for row in tabla.find_all('tr'):
                cols = row.find_all('td')
                if len(cols) == 2:
                    key = cols[0].text.strip().upper()
                    value = cols[1].text.strip()
                    info[key] = value

        coords_raw = placemark.find('.//kml:coordinates', ns).text.strip()
        coords = [(float(c.split(',')[0]), float(c.split(',')[1])) for c in coords_raw.split()]
        coords = cerrar_poligono(coords)

        try:
            poly = Polygon(coords).simplify(0.0005, preserve_topology=True)
            if poly.is_empty or not poly.is_valid:
                continue
            info['coordenadas'] = list(poly.exterior.coords)
        except TopologicalError:
            continue

        # Generar ID único
        id_predio = info.get("ID_PREDIO", "X")
        id_unico = info.get("ID_UNICO", "Y")
        info["ID"] = str(id_predio) + str(id_unico)

        terrenos.append(info)

# Ejecutar procesamiento
for file in kml_files:
    procesar_kml(file)

# DataFrame con polígonos
df = pd.DataFrame(terrenos)

# Enlazar con datos de especies por ID_ESP1
df = df.merge(df_especies, on="ID_ESP1", how="left")

# Construir GeoJSON enriquecido
features = []
for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "id": row["ID"],  # ID externo para plotly
        "properties": {
            "ID_ESP1": row["ID_ESP1"],
            "ID_es":row.get("ID_ESPECIE_GLOBAL"),
            "Nombre común": row.get("Nombre común"),
            "Nombre científico": row.get("Nombre científico"),
            "Clasificación": row.get("Clasificación"),
            "Superficie":row.get("SUP_HA")
            #"Nombre originario": row.get("Nombre originario")
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [row["coordenadas"]]
        }
    }
    features.append(feature)

geojson = {"type": "FeatureCollection", "features": features}

# Guardar archivo
with open("terrenos_enriquecido_ID.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("GeoJSON enriquecido generado correctamente.")


GeoJSON enriquecido generado correctamente.


In [15]:
df.head(10)

,nombre,FID,ID_UNICO,ID_PREDIO,NOMBRE,ID_USO,ID_SUBUSO,ID_ESTRUC,ID_TIFO,ID_STIF,...,OBS,SUP_HA,coordenadas,ID,index,ID_ESPECIE_GLOBAL,Nombre científico,Nombre común,Clasificación,Nombre originario
0,02,0,1,02,Las Palmas,06,07,00,,,...,,0.16,"[(-73.14953102003405, -39.74768325608614), (-7...",021,NaN,NaN,NaN,NaN,NaN,NaN
1,02,1,2,02,Las Palmas,06,07,00,,,...,,0.38,"[(-73.14092003614246, -39.74232332453637), (-7...",022,NaN,NaN,NaN,NaN,NaN,NaN
2,02,2,3,02,Las Palmas,06,07,00,,,...,,0.56,"[(-73.14718750824242, -39.73933057296891), (-7...",023,NaN,NaN,NaN,NaN,NaN,NaN
3,02,3,4,02,Las Palmas,06,07,00,,,...,,0.03,"[(-73.14264311204646, -39.74138113809885), (-7...",024,NaN,NaN,NaN,NaN,NaN,NaN
4,02,4,5,02,Las Palmas,06,07,00,,,...,,0.4,"[(-73.14439251501709, -39.73998284468944), (-7...",025,NaN,NaN,NaN,NaN,NaN,NaN
5,02,5,6,02,Las Palmas,04,03,02,,,...,,0.49,"[(-73.14350630915092, -39.73988627391925), (-7...",026,19.0,20.0,Nothofagus dombeyi,coihue,Nativa,Koiwe
6,02,6,7,02,Las Palmas,04,01,01,,,...,,1.58,"[(-73.14845294531658, -39.73687550753437), (-7...",027,9.0,10.0,Cupressus lusitanica,cipres,Exótica,
7,02,7,8,02,Las Palmas,04,02,02,09,03,...,,0.56,"[(-73.14845294531658, -39.73687550753437), (-7...",028,21.0,22.0,Nothofagus obliqua,roble,Nativa,koyam
8,02,8,9,02,Las Palmas,04,02,02,09,02,...,,0.43,"[(-73.15919708871994, -39.76249516120033), (-7...",029,19.0,20.0,Nothofagus dombeyi,coihue,Nativa,Koiwe
9,02,9,10,02,Las Palmas,04,01,01,,,...,,0.78,"[(-73.15830049886726, -39.76184195592293), (-7...",0210,24.0,25.0,Pseudotsuga menziesii,Pino oregon,Exótica,


Agregar limites y simplificar

In [4]:
from shapely.geometry import Polygon, mapping
from shapely.ops import unary_union
import zipfile, xml.etree.ElementTree as ET, os, json

def extraer_poligonos_desde_kmz(kmz_path):
    extract_dir = kmz_path.replace(".kmz", "_extraido")
    with zipfile.ZipFile(kmz_path, 'r') as kmz:
        kmz.extractall(extract_dir)
    kml_path = os.path.join(extract_dir, "doc.kml")
    tree = ET.parse(kml_path)
    root = tree.getroot()
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    coords_list = []
    for placemark in root.findall('.//kml:Placemark', ns):
        coords_element = placemark.find('.//kml:coordinates', ns)
        if coords_element is not None:
            coords_raw = coords_element.text.strip()
            coords = [(float(c.split(',')[0]), float(c.split(',')[1])) for c in coords_raw.split()]
            if coords[0] != coords[-1]:
                coords.append(coords[0])
            coords_list.append(Polygon(coords))
    return coords_list

# Lista de archivos KMZ
kmz_paths = ["Limite_Las_Palmas.kmz", 'Limite_Los_Pinos.kmz']
tolerancia = 0.0005

# Extraer y unificar
todos_los_poligonos = []
for path in kmz_paths:
    todos_los_poligonos += extraer_poligonos_desde_kmz(path)

unificado = unary_union([p.simplify(tolerancia, preserve_topology=True) for p in todos_los_poligonos])

# Exportar GeoJSON
geojson = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "id": "Limite_Unificado",
        "properties": {"nombre": "Predios Unificados"},
        "geometry": mapping(unificado)
    }]
}

with open("limites_unificados.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)
